In [1]:
import requests
from bs4 import BeautifulSoup
import time

# 1페이지 URL
base_url = "https://archives.warmemo.or.kr:8443"
list_url = f"{base_url}/pss/media/pssMediaList.do?MID=UM00015&ctgryClfcCd=10"

# 결과 저장용 리스트
results = []

# 1. 리스트 페이지 요청
res = requests.get(list_url)
soup = BeautifulSoup(res.text, 'html.parser')

# 2. 무기 상세 페이지 링크 수집
items = soup.select("div.data-wrap figure > figcaption h4 a")

for item in items:
    href = item['href']
    detail_url = base_url + href
    
    try:
        detail_res = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_res.text, 'html.parser')

        name = detail_soup.select_one("div.data-detail-header h2").get_text(strip=True)
        content = detail_soup.select_one("p.content").get_text(strip=True)

        results.append({
            "용어": name,
            "내용": content
        })
        print(f"✅ 수집: {name}")
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
    time.sleep(0.5)  # 과부하 방지

# 결과만 리턴
results


✅ 수집: C-123J 수송기
✅ 수집: 500MD 다목적 헬기
✅ 수집: C-119G 수송기
✅ 수집: B-52D 전략폭격기
✅ 수집: UH-1H 다용도 헬기
✅ 수집: T-103 훈련기
✅ 수집: 선양 J-6전투기
✅ 수집: 1990년 기증유물(꽂을대/집)
✅ 수집: 1990년 기증유물(터키 사제 권총)
✅ 수집: 1990년 기증유물(의류대(衣類袋))
✅ 수집: 1990년 기증유물(서류가방)
✅ 수집: 1989년 기증유물(F-51 전투기(Mustang))
✅ 수집: C-123J 수송기
✅ 수집: 500MD 다목적 헬기
✅ 수집: C-119G 수송기
✅ 수집: B-52D 전략폭격기
✅ 수집: UH-1H 다용도 헬기
✅ 수집: T-103 훈련기
✅ 수집: 선양 J-6전투기
✅ 수집: 1990년 기증유물(꽂을대/집)
✅ 수집: 1990년 기증유물(터키 사제 권총)
✅ 수집: 1990년 기증유물(의류대(衣類袋))
✅ 수집: 1990년 기증유물(서류가방)
✅ 수집: 1989년 기증유물(F-51 전투기(Mustang))


[{'용어': 'C-123J 수송기',
  '내용': 'C-123의 개발은 제2차 세계대전 이후 미 공군(USAF)이 단거리 이착륙(STOL) 능력을 갖춘 전술 수송기를 필요로 하면서 시작되었다. 1948년 미국 체이스(Chase Aircraft) 사가 개발 중이던 XCG-20 글라이더를 기반으로 1950년 XC-123 및 XC-123A 시제기를 제작하였으나, 경영 문제로 인해 1953년 미국 페어차일드(Fairchild Aircraft) 사가 생산권을 인수하여 양산형인 C-123B를 최초로 실전 배치하였다. C-123J는 극저온 환경에서 운용할 수 있도록 C-123B를 개량한 모델로, 엔진과 기체를 저온 환경에서도 작동 가능하도록 개조하였으며, 랜딩기어에 탈부착 가능한 스키 키트를 장착할 수 있어 미 공군의 북극 지역 기지 보급 임무에 활용되었다. 대한민국 공군은 노후화된 C-46D 및 C-54 기종을 대체하기 위해 미국으로부터 C-123K와 C-123J를 인수하여 운용하였다. 도입된 C-123은 병력 및 물자 수송, 야간 조명탄 투하 임무 등에 활용되었으며, C-130이 본격적으로 도입되기 전까지 한국 공군의 주요 전술 수송기로 운용되었다. 이후 1994년 12월 전량 퇴역하였다.'},
 {'용어': '500MD 다목적 헬기',
  '내용': '미국 맥도널 더글라스(McDonnell Douglas)사에서 개발한 다목적 경량 헬리콥터이다. 휴즈(Hughes)사에서 개발한 민간용 헬기인 휴즈-500(Hughes-500)을 군용으로 개량한 것이다. 1970년대 초반, 대한민국은 북한의 기갑전력에 대응할 공격형 헬기가 부족하여 미국의 500MD의 도입이 적합하다고 판단하였다. 다만 단순 수입이 아닌 국내 항공산업 발전과 기술 이전을 고려하여 1976년 라이선스 계약을 체결, 대한항공이 직접 500MD 헬리콥터를 생산할 수 있도록 하였다. 500MD는 약 250대가 제작되어 주로 육군과 해군에서 다양한 용도로 활용되었고 2017년 8월 1일 이후 최신 기종으로 점진

In [4]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')  # 화면 표시
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# 무기 상세 페이지 정보 추출
def get_weapon_detail(driver, url):
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.data-detail-header h2'))
    )
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    title = soup.select_one('div.data-detail-header h2').text.strip()
    content_tag = soup.select_one('p.content')
    content = content_tag.text.strip() if content_tag else ''
    return {'용어': title, '내용': content}

# 전체 페이지 크롤링 (마지막 페이지 도달 시 자동 종료)
def crawl_all_pages(save_csv='전쟁기념관_무기및장비.csv'):
    driver = init_driver()
    base_url = "https://archives.warmemo.or.kr:8443"
    start_url = base_url + "/pss/media/pssMediaList.do?MID=UM00015&ctgryClfcCd=10"
    driver.get(start_url)

    all_data = []
    current_page = 1

    while True:
        print(f"📄 {current_page} 페이지 크롤링 중...")
        try:
            driver.execute_script(f"fnGoPage({current_page});")
            time.sleep(2)
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.col-xs-6.col-sm-4.col-md-3'))
            )
        except:
            print("⛔️ 더 이상 페이지가 없습니다. 종료합니다.")
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        items = soup.select('div.col-xs-6.col-sm-4.col-md-3')

        if not items:
            print("⛔️ 더 이상 아이템이 없습니다.")
            break

        for item in items:
            a_tag = item.select_one('a.thumb')
            if not a_tag:
                continue
            href = a_tag['href']
            full_url = base_url + href
            try:
                data = get_weapon_detail(driver, full_url)
                all_data.append(data)
                print(f"✅ {data['용어']}")
            except Exception as e:
                print(f"❌ 상세 페이지 오류: {e}")
            finally:
                driver.back()
                time.sleep(1)

        current_page += 1

    df = pd.DataFrame(all_data)
    df.to_csv(save_csv, index=False, encoding='utf-8-sig')
    print(f"\n🎉 전체 완료! 총 {len(all_data)}개 저장됨.")
    driver.quit()


In [5]:
crawl_all_pages()

📄 1 페이지 크롤링 중...
✅ C-123J 수송기
✅ 500MD 다목적 헬기
✅ C-119G 수송기
✅ B-52D 전략폭격기
✅ UH-1H 다용도 헬기
✅ T-103 훈련기
✅ 선양 J-6전투기
✅ 1990년 기증유물(꽂을대/집)
✅ 1990년 기증유물(터키 사제 권총)
✅ 1990년 기증유물(의류대(衣類袋))
✅ 1990년 기증유물(서류가방)
✅ 1989년 기증유물(F-51 전투기(Mustang))
📄 2 페이지 크롤링 중...
✅ 1989년 기증유물(M1 90mm 고사포)
✅ 1989년 기증유물(3.5인치 로켓포(M20A1B1))
✅ 1991년 차입유물(M1917A1 브라우닝 기관총_몸통)
✅ 1992년 구입유물(122mm 곡사포 M1938(M-30))
✅ 1992년 구입유물(76.2mm 사단포 M1942)
✅ 1993년 이관유물(M18 57mm 무반동총)
✅ 1993년 기증유물(7.7mm 브렌 경기관총)
✅ 1994년 기증유물(9mm 오웬 기관단총)
✅ 1994년 기증유물(9mm 오웬 기관단총-탄창)
✅ 1993년 기증유물(T-6 훈련기(건국기))
✅ 1993년 기증유물(F-86F 전투기)
✅ 1990년 기증유물(M4A3E8 전차(셔먼전차))
📄 3 페이지 크롤링 중...
✅ 1991년 기증유물(M46 전차(패튼전차))
✅ 1992년 기증유물(YAK-18 훈련기)
✅ 1992년 기증유물(T-34 전차 (1))
✅ 1992년 기증유물(T-34 전차 (2))
✅ 1992년 기증유물(MIG-15 전투기)
✅ 1992년 기증유물(120mm 박격포)
✅ 1992년 기증유물(37mm 자동고사포)
✅ 1992년 기증유물(85mm 고사포)
✅ 1990년 기증유물(O-1A 관측기(L-19))
✅ 1992년 기증유물(M-36 전차)
✅ 1992년 기증유물(SU-100 자주포)
✅ 1992년 기증유물(LVT-3C 상륙장갑차)
📄 4 페이지 크롤링 중...
✅ 1992년 기증유물(M114A1 155mm 곡사포)
✅ 1992년 기증유물(BM13 132㎜ 방사포)
✅ 2000

In [26]:
import re
import pandas as pd

# ✅ 용어 전처리 함수
def preprocess_term(term: str) -> str:
    keyword_list = ['기증유물', '차입유물', '구입유물', '이관유물']
    if any(kw in term for kw in keyword_list):
        # 바깥 괄호 내용 추출 (가장 마지막 괄호 안만 가져오기)
        matches = re.findall(r'\(([^()]*(?:\([^()]*\)[^()]*)*)\)', term)
        if matches:
            return matches[-1].strip()
        else:
            return term.strip()
    else:
        # 일반 용어는 소괄호, 대괄호 제거
        term = re.sub(r'\(.*?\)', '', term)
        term = re.sub(r'\[.*?\]', '', term)
        return term.strip()

# ✅ 내용 전처리 함수 (숫자 마침표 보존)
def preprocess_content(content: str) -> str:
    sentence_end_pattern = re.compile(r'(?<!\d)\.(?!\d)')  # 숫자 사이가 아닌 마침표만 기준
    parts = sentence_end_pattern.split(content)
    return parts[0].strip()

# ✅ 데이터 불러오기
df = pd.read_csv("전쟁기념관_무기및장비.csv")

# ✅ 전처리 적용
df["용어"] = df["용어"].apply(preprocess_term)
df["내용"] = df["내용"].apply(preprocess_content)

# ✅ 내용이 완전히 같은 중복 용어 제거 (예: T-34 전차(2))
df.drop_duplicates(subset=["용어", "내용"], keep='first', inplace=True)

# ✅ 전처리 결과 저장
df.to_csv("전쟁기념관_무기및장비_전처리.csv", index=False, encoding='utf-8-sig')
print(f"🎉 전처리 완료! 총 {len(df)}개 항목이 저장되었습니다.")


🎉 전처리 완료! 총 283개 항목이 저장되었습니다.


In [27]:
wp = pd.read_csv('전쟁기념관_무기및장비_전처리.csv')
wp

,용어,내용
0,C-123J 수송기,C-123의 개발은 제2차 세계대전 이후 미 공군(USAF)이 단거리 이착륙(STO...
1,500MD 다목적 헬기,미국 맥도널 더글라스(McDonnell Douglas)사에서 개발한 다목적 경량 헬...
2,C-119G 수송기,C-119 플라잉 박스카는 미국 페어차일드(Fairchild Aircraft)사에서...
3,B-52D 전략폭격기,"미국 보잉(Boeing)사에서 개발한 전략 폭격기로, 제2차 세계대전 종전 직후 전..."
4,UH-1H 다용도 헬기,미국 벨(Bell)사에서 개발한 다목적 헬리콥터이다
...,...,...
278,고소총통,고소총통은 고려시대 총통으로 해당 유물은 경희대학교 박물관 소장품을 복제한 것이다
279,김성일 철퇴,철퇴는 조선시대 때 의식용으로 사용하던 무기이다
280,투구,투구는 적의 무기로부터 머리를 보호하기 위해 머리에 쓰던 쇠모자이다
281,청동 화살촉,청동 화살촉은 청동기시대의 원거리 살상무기이다


In [30]:
# 기존 전처리된 데이터 불러오기
wp = pd.read_csv("전쟁기념관_무기및장비_전처리.csv")

# 1. '말재갈 2' 제거
wp = wp[wp["용어"] != "말재갈 2"]

# 2. 'T-34 전차 (2)' 제거
wp = wp[wp["용어"] != "T-34 전차 (2)"]

# 3. 'T-34 전차 (1)' → 'T-34 전차' 로 변경
wp["용어"] = wp["용어"].replace("T-34 전차 (1)", "T-34 전차")

# 결과 저장
wp.to_csv("전쟁기념관_무기및장비_전처리.csv", index=False, encoding='utf-8-sig')
print(f"✅ 최종 정리 완료! 총 {len(wp)}개 항목이 저장되었습니다.")


✅ 최종 정리 완료! 총 281개 항목이 저장되었습니다.


In [24]:
import pandas as pd

# CSV 불러오기
wp = pd.read_csv('전쟁기념관_무기및장비_전처리.csv')

# 특정 용어 기준으로 중복 제거: 첫 번째만 남기고 나머지 제거
target_term = "Colt M1911A1 자동권총"
wp = wp[~((wp["용어"] == target_term) & (wp.duplicated(subset=["용어"], keep='first')))]

# 저장 (선택)
wp.to_csv("전처리_중복제거.csv", index=False, encoding='utf-8-sig')
print("✅ 두 번째 중복 용어 제거 완료!")


✅ 두 번째 중복 용어 제거 완료!


In [25]:
wp

,용어,내용
0,C-123J 수송기,C-123의 개발은 제2차 세계대전 이후 미 공군(USAF)이 단거리 이착륙(STO...
1,500MD 다목적 헬기,미국 맥도널 더글라스(McDonnell Douglas)사에서 개발한 다목적 경량 헬...
2,C-119G 수송기,C-119 플라잉 박스카는 미국 페어차일드(Fairchild Aircraft)사에서...
3,B-52D 전략폭격기,"미국 보잉(Boeing)사에서 개발한 전략 폭격기로, 제2차 세계대전 종전 직후 전..."
4,UH-1H 다용도 헬기,미국 벨(Bell)사에서 개발한 다목적 헬리콥터이다
...,...,...
278,고소총통,고소총통은 고려시대 총통으로 해당 유물은 경희대학교 박물관 소장품을 복제한 것이다
279,김성일 철퇴,철퇴는 조선시대 때 의식용으로 사용하던 무기이다
280,투구,투구는 적의 무기로부터 머리를 보호하기 위해 머리에 쓰던 쇠모자이다
281,청동 화살촉,청동 화살촉은 청동기시대의 원거리 살상무기이다
